In [1]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, DefineVariable
from biogeme.models import loglogit

In [9]:
pandas = pd.read_table("airline.dat")
database = db.Database("airline",pandas)
pd.options.display.float_format = '{:.3g}'.format

globals().update(database.variables)

# Exclude
exclude = ((ArrivalTimeHours_1   ==  -1)  | (Cont_Income==-1))
database.remove(exclude)
  
# exclude = (Cont_Income==-1)
# database.remove(exclude)
# Choice
chosenAlternative = ( (  BestAlternative_1   *  1  ) + (  BestAlternative_2   *  2  ) ) + (  BestAlternative_3   *  3  )

In [6]:
database.variables

{'AirlineFirstFlight_1': AirlineFirstFlight_1,
 'AirlineFirstFlight_2': AirlineFirstFlight_2,
 'AirlineFirstFlight_3': AirlineFirstFlight_3,
 'AirlineSecondFlight_1': AirlineSecondFlight_1,
 'AirlineSecondFlight_2': AirlineSecondFlight_2,
 'AirlineSecondFlight_3': AirlineSecondFlight_3,
 'AirplaneFirstFlight_1': AirplaneFirstFlight_1,
 'AirplaneFirstFlight_2': AirplaneFirstFlight_2,
 'AirplaneFirstFlight_3': AirplaneFirstFlight_3,
 'AirplaneSecondFlight_1': AirplaneSecondFlight_1,
 'AirplaneSecondFlight_2': AirplaneSecondFlight_2,
 'AirplaneSecondFlight_3': AirplaneSecondFlight_3,
 'ArrivalTimeHours_1': ArrivalTimeHours_1,
 'ArrivalTimeHours_2': ArrivalTimeHours_2,
 'ArrivalTimeHours_3': ArrivalTimeHours_3,
 'ArrivalTimeMins_1': ArrivalTimeMins_1,
 'ArrivalTimeMins_2': ArrivalTimeMins_2,
 'ArrivalTimeMins_3': ArrivalTimeMins_3,
 'BestAlternative_1': BestAlternative_1,
 'BestAlternative_2': BestAlternative_2,
 'BestAlternative_3': BestAlternative_3,
 'Cont_Income': Cont_Income,
 'Depart

In [10]:
#Parameters to be estimated
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
Constant1= Beta('Constant1',0,None,None,1)
Constant2= Beta('Constant2',0,None,None,0)
Constant3= Beta('Constant3',0,None,None,0)
Fare= Beta('Fare',0,None,None,0)
Legroom = Beta('Legroom',0,None,None,0)
SchedDE = Beta('SchedDE',0,None,None,0)
SchedDL = Beta('SchedDL',0,None,None,0)
Total_TT= Beta('Total_TT',0,None,None,0)
Income1 = Beta('Income1',0,None,None,1)
Income2 = Beta('Income2',0,None,None,0)
Income3 = Beta('Income3',0,None,None,0)


# Define here arithmetic expressions for name that are not directly
# available from the data

DepartureTimeSensitive  = DefineVariable('DepartureTimeSensitive', q11_DepartureOrArrivalIsImportant   ==  1 ,
                                         database)
ArrivalTimeSensitive  = DefineVariable('ArrivalTimeSensitive', q11_DepartureOrArrivalIsImportant   ==  2 ,database)
Missing  = DefineVariable('Missing',(  q11_DepartureOrArrivalIsImportant   !=  1  ) *\
                          (  q11_DepartureOrArrivalIsImportant   !=  2  ),database)
DesiredDepartureTime  = DefineVariable('DesiredDepartureTime',q12_IdealDepTime ,database)
DesiredArrivalTime  = DefineVariable('DesiredArrivalTime',q13_IdealArrTime ,database)
ScheduledDelay_1  = DefineVariable('ScheduledDelay_1',(  DepartureTimeSensitive   *\
                                                       (  DepartureTimeMins_1   -  DesiredDepartureTime   ) ) +\
                                   (  ArrivalTimeSensitive   * (  ArrivalTimeMins_1   -  DesiredArrivalTime   ) ),
                                   database)
ScheduledDelay_2  = DefineVariable('ScheduledDelay_2',(  DepartureTimeSensitive   * (  DepartureTimeMins_2   -\
                                                                                     DesiredDepartureTime   ) ) +\
                                   (  ArrivalTimeSensitive   * (  ArrivalTimeMins_2   -  DesiredArrivalTime   ) ),
                                   database)
ScheduledDelay_3  = DefineVariable('ScheduledDelay_3',(  DepartureTimeSensitive   *\
                                                       (  DepartureTimeMins_3   -  DesiredDepartureTime   ) ) +\
                                   (  ArrivalTimeSensitive   * (  ArrivalTimeMins_3   -  DesiredArrivalTime   ) ),
                                   database)
Opt1_SchedDelayEarly  = DefineVariable('Opt1_SchedDelayEarly',(  -(ScheduledDelay_1 )  *\
                                                               (  ScheduledDelay_1   <  0  ) ) /  60 ,database)
Opt2_SchedDelayEarly  = DefineVariable('Opt2_SchedDelayEarly',(  -(ScheduledDelay_2 )  *\
                                                               (  ScheduledDelay_2   <  0  ) ) /  60 ,database)
Opt3_SchedDelayEarly  = DefineVariable('Opt3_SchedDelayEarly',(  -(ScheduledDelay_3 )  *\
                                                               (  ScheduledDelay_3   <  0  ) ) /  60 ,database)
Opt1_SchedDelayLate  = DefineVariable('Opt1_SchedDelayLate',(  ScheduledDelay_1   *\
                                                             (  ScheduledDelay_1   >  0  ) ) /  60 ,database)
Opt2_SchedDelayLate  = DefineVariable('Opt2_SchedDelayLate',(  ScheduledDelay_2   *\
                                                             (  ScheduledDelay_2   >  0  ) ) /  60 ,database)
Opt3_SchedDelayLate  = DefineVariable('Opt3_SchedDelayLate',(  ScheduledDelay_3   *\
                                                             (  ScheduledDelay_3   >  0  ) ) /  60 ,database)

# Utilities
Opt1 = Constant1 + Fare * Fare_1 + Legroom * Legroom_1 + SchedDE * Opt1_SchedDelayEarly +\
SchedDL * Opt1_SchedDelayLate + Total_TT * TripTimeHours_1 + Income1*Cont_Income
Opt2 = Constant2 + Fare * Fare_2 + Legroom * Legroom_2 + SchedDE * Opt2_SchedDelayEarly +\
SchedDL * Opt2_SchedDelayLate + Total_TT * TripTimeHours_2 + Income2*Cont_Income
Opt3 = Constant3 + Fare * Fare_3 + Legroom * Legroom_3 + SchedDE * Opt3_SchedDelayEarly +\
SchedDL * Opt3_SchedDelayLate + Total_TT * TripTimeHours_3 + Income3*Cont_Income
V = {1: Opt1,2: Opt2,3: Opt3}
av = {1: 1,2: 1,3: 1}

# The choice model is a logit, with availability conditions
logprob = loglogit(V,av,chosenAlternative)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_airline_socioecon"
results = biogeme.estimate()
# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")

             Value  Std err  t-test  p-value  Rob. Std err  Rob. t-test  \
Constant2    -1.12    0.151   -7.43 1.13e-13         0.147         -7.6   
Constant3    -0.99    0.156   -6.36 2.01e-10         0.156        -6.35   
Fare       -0.0196 0.000747   -26.2        0      0.000861        -22.7   
Income2   -0.00104 0.000646    -1.6    0.109      0.000665        -1.56   
Income3   -0.00462 0.000811    -5.7  1.2e-08      0.000885        -5.22   
Legroom      0.219   0.0269    8.12 4.44e-16        0.0287         7.64   
SchedDE     -0.139   0.0166   -8.32        0        0.0173        -7.99   
SchedDL     -0.094   0.0136   -6.91 4.92e-12        0.0146        -6.44   
Total_TT    -0.339   0.0717   -4.74 2.19e-06        0.0719        -4.72   

           Rob. p-value  
Constant2      3.09e-14  
Constant3      2.13e-10  
Fare                  0  
Income2           0.119  
Income3        1.76e-07  
Legroom        2.24e-14  
SchedDE        1.33e-15  
SchedDL        1.22e-10  
Total_TT       